In [1]:
from tensorflow import keras


In [2]:
#Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/Attention_Test"
Model_path = "E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/Attention_Test"

In [4]:
from CustomLayers import *

In [5]:
model_folder = "smut_MultiHeadAttentionLNN_1"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention})

ValueError: in user code:

    E:\learning resource\PhD\PHD_Notebook\Code\ML_composer\CustomLayers.py:227 call  *
        query = tf.einsum('bsd,dd->bsd',X,self.wq)
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\special_math_ops.py:684 einsum
        return _einsum_v2(equation, *inputs, **kwargs)
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\special_math_ops.py:1113 _einsum_v2
        return gen_linalg_ops.einsum(inputs, resolved_equation)
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\gen_linalg_ops.py:1104 einsum
        "Einsum", inputs=inputs, equation=equation, name=name)
    C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\op_def_library.py:744 _apply_op_helper
        attrs=attr_prot

In [ ]:
model.summary()